In [1]:
# ==================== IMPORTS ====================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from copy import deepcopy
import kagglehub
import os
import json
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import glob

print("✓ All libraries imported successfully!")

# Configuration
RANDOM_SEEDS = [42, 123, 456, 789, 1011]
N_SPLITS = 5
NUM_CLIENTS = 5
LOCAL_EPOCHS = 5
GLOBAL_ROUNDS = 20
BATCH_SIZE = 64
LEARNING_RATE = 0.001

print("\nConfiguration:")
print(f"  Random seeds: {RANDOM_SEEDS}")
print(f"  K-fold splits: {N_SPLITS}")
print(f"  Total evaluations per config: {len(RANDOM_SEEDS)} × {N_SPLITS} = {len(RANDOM_SEEDS) * N_SPLITS}")

c:\Users\almir\ai-privacy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ All libraries imported successfully!

Configuration:
  Random seeds: [42, 123, 456, 789, 1011]
  K-fold splits: 5
  Total evaluations per config: 5 × 5 = 25


In [2]:
# ==================== LOAD DIABETES DATASET ====================
print("\n" + "="*80)
print("LOADING DIABETES DATASET FROM KAGGLE")
print("="*80)

try:
    diabetes_paths = glob.glob('/kaggle/input/*/diabetes_binary_health_indicators_BRFSS2015.csv')
    
    if diabetes_paths:
        diabetes_csv = diabetes_paths[0]
        print("✓ Using Kaggle native dataset path")
    else:
        raise FileNotFoundError("Dataset not found in /kaggle/input/")
        
except (FileNotFoundError, IndexError):
    print("✓ Using kagglehub for dataset download")
    diabetes_path = kagglehub.dataset_download("alexteboul/diabetes-health-indicators-dataset")
    diabetes_csv = f"{diabetes_path}/diabetes_binary_health_indicators_BRFSS2015.csv"

df_diabetes = pd.read_csv(diabetes_csv)
print(f"✓ Diabetes dataset loaded: {df_diabetes.shape}")


LOADING DIABETES DATASET FROM KAGGLE
✓ Using kagglehub for dataset download
✓ Diabetes dataset loaded: (253680, 22)


In [3]:
# ==================== PREPROCESS DATA ====================
print("\n" + "="*80)
print("PREPROCESSING DATA")
print("="*80)

X_diabetes = df_diabetes.drop(columns=['Diabetes_binary']).values
y_diabetes = df_diabetes['Diabetes_binary'].values
print(f"✓ Diabetes - Features: {X_diabetes.shape}, Target: {y_diabetes.shape}")


PREPROCESSING DATA
✓ Diabetes - Features: (253680, 21), Target: (253680,)


In [4]:
# ==================== MODEL ARCHITECTURES ====================

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, output_size=2):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return self.linear(x)

class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128, 64], output_size=2, dropout_rate=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

print("✓ Model architectures defined")

✓ Model architectures defined


In [5]:
# ==================== FEDERATED LEARNING UTILITIES ====================

def fedavg_aggregate(client_models, client_data_sizes):
    """FedAvg: Weighted average based on client data sizes"""
    total_size = sum(client_data_sizes)
    global_state = {}
    
    for key in client_models[0].state_dict().keys():
        global_state[key] = sum(
            client_models[i].state_dict()[key] * (client_data_sizes[i] / total_size)
            for i in range(len(client_models))
        )
    
    return global_state

def fedprox_aggregate(client_models, client_data_sizes):
    """FedProx: Similar to FedAvg"""
    return fedavg_aggregate(client_models, client_data_sizes)

def qfedavg_aggregate(client_models, client_losses, q=0.2):
    """q-FedAvg: Fairness-weighted aggregation"""
    lipschitz = [1.0 / (loss + 1e-10) for loss in client_losses]
    weights = [l ** q for l in lipschitz]
    total_weight = sum(weights)
    
    global_state = {}
    for key in client_models[0].state_dict().keys():
        global_state[key] = sum(
            client_models[i].state_dict()[key] * (weights[i] / total_weight)
            for i in range(len(client_models))
        )
    
    return global_state

def scaffold_aggregate(client_models, client_data_sizes):
    """SCAFFOLD: Simplified version"""
    return fedavg_aggregate(client_models, client_data_sizes)

# FedAdam state
fedadam_state = {'m': None, 'v': None, 't': 0}

def fedadam_aggregate(client_models, client_data_sizes, beta1=0.9, beta2=0.999, eta=0.01, tau=1e-3):
    """FedAdam: Adaptive federated optimization"""
    total_size = sum(client_data_sizes)
    avg_state = {}
    
    device = next(client_models[0].parameters()).device
    
    for key in client_models[0].state_dict().keys():
        avg_state[key] = sum(
            client_models[i].state_dict()[key] * (client_data_sizes[i] / total_size)
            for i in range(len(client_models))
        )
    
    if fedadam_state['m'] is None:
        fedadam_state['m'] = {key: torch.zeros_like(val).to(device) for key, val in avg_state.items()}
        fedadam_state['v'] = {key: torch.zeros_like(val).to(device) for key, val in avg_state.items()}
    
    fedadam_state['t'] += 1
    global_state = {}
    
    for key in avg_state.keys():
        delta = avg_state[key]
        fedadam_state['m'][key] = fedadam_state['m'][key].to(device)
        fedadam_state['v'][key] = fedadam_state['v'][key].to(device)
        
        fedadam_state['m'][key] = beta1 * fedadam_state['m'][key] + (1 - beta1) * delta
        fedadam_state['v'][key] = beta2 * fedadam_state['v'][key] + (1 - beta2) * (delta ** 2)
        
        m_hat = fedadam_state['m'][key] / (1 - beta1 ** fedadam_state['t'])
        v_hat = fedadam_state['v'][key] / (1 - beta2 ** fedadam_state['t'])
        
        global_state[key] = avg_state[key] + eta * m_hat / (torch.sqrt(v_hat) + tau)
    
    return global_state

def train_fl_client(client_model, train_loader, global_model=None, epochs=5, lr=0.001, mu=0.01, use_proximal=False):
    """Train a single FL client"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    client_model = client_model.to(device)
    
    if use_proximal and global_model is not None:
        global_model = global_model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(client_model.parameters(), lr=lr)
    
    client_model.train()
    total_loss = 0.0
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = client_model(batch_X)
            loss = criterion(outputs, batch_y)
            
            if use_proximal and global_model is not None:
                proximal_term = 0.0
                for w, w_t in zip(client_model.parameters(), global_model.parameters()):
                    proximal_term += (w - w_t).norm(2)
                loss += (mu / 2) * proximal_term
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        total_loss += epoch_loss / len(train_loader)
    
    return client_model, total_loss / epochs

def distribute_data_to_clients(X, y, num_clients, batch_size):
    """Distribute data IID to clients"""
    dataset = TensorDataset(torch.FloatTensor(X), torch.LongTensor(y))
    
    total_size = len(dataset)
    client_sizes = [total_size // num_clients] * num_clients
    client_sizes[-1] += total_size % num_clients
    
    indices = torch.randperm(total_size).tolist()
    client_loaders = []
    start_idx = 0
    
    for size in client_sizes:
        client_indices = indices[start_idx:start_idx + size]
        client_dataset = Subset(dataset, client_indices)
        client_loader = DataLoader(client_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        client_loaders.append(client_loader)
        start_idx += size
    
    return client_loaders, client_sizes

def evaluate_model(model, X, y):
    """Evaluate model and return accuracy, f1"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    X_tensor = torch.FloatTensor(X).to(device)
    
    with torch.no_grad():
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs, 1)
    
    accuracy = accuracy_score(y, predicted.cpu().numpy())
    f1 = f1_score(y, predicted.cpu().numpy(), average='weighted', zero_division=0)
    
    return accuracy, f1

print("✓ FL utilities defined")

✓ FL utilities defined


In [ ]:
# ==================== FL CONTINUATION - DIABETES FNN ONLY ====================
print("\n" + "="*80)
print("FEDERATED LEARNING - DIABETES FNN CONTINUATION")
print("="*80)

try:
    models_dir = "/kaggle/working/models_fl_diabetes_continue"
    os.makedirs(models_dir, exist_ok=True)
except:
    models_dir = os.path.join(r"c:\Users\almir\ai-privacy\backend", "models_fl_diabetes_continue")
    os.makedirs(models_dir, exist_ok=True)

print(f"\nResults will be saved to: {models_dir}")

# Only run FNN with remaining aggregations
MODEL_TYPE = 'FNN'
REMAINING_AGGREGATIONS = ['q-FedAvg', 'SCAFFOLD', 'FedAdam']

print(f"\nConfiguration: Diabetes FNN - 3 aggregation methods")
print(f"  Remaining: {REMAINING_AGGREGATIONS}")

fl_results = {}

for agg_method in REMAINING_AGGREGATIONS:
    print(f"\n  Aggregation: {agg_method}")
    
    all_accuracies = []
    all_f1s = []
    
    # Reset FedAdam state
    fedadam_state['m'] = None
    fedadam_state['v'] = None
    fedadam_state['t'] = 0
    
    for run_idx, seed in enumerate(RANDOM_SEEDS):
        skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=seed)
        
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_diabetes, y_diabetes)):
            X_train, X_val = X_diabetes[train_idx], X_diabetes[val_idx]
            y_train, y_val = y_diabetes[train_idx], y_diabetes[val_idx]
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            
            # Distribute data to clients
            client_loaders, client_sizes = distribute_data_to_clients(
                X_train_scaled, y_train, NUM_CLIENTS, BATCH_SIZE
            )
            
            # Initialize global model (FNN)
            input_size = X_train_scaled.shape[1]
            global_model = FeedforwardNN(input_size, hidden_sizes=[128, 64], output_size=2)
            
            # FL Training
            for round_num in range(GLOBAL_ROUNDS):
                client_models = []
                client_losses = []
                
                for client_id in range(NUM_CLIENTS):
                    client_model = deepcopy(global_model)
                    use_proximal = (agg_method == 'FedProx')
                    
                    trained_model, loss = train_fl_client(
                        client_model, client_loaders[client_id],
                        global_model=global_model if use_proximal else None,
                        epochs=LOCAL_EPOCHS, lr=LEARNING_RATE, use_proximal=use_proximal
                    )
                    
                    client_models.append(trained_model)
                    client_losses.append(loss)
                
                # Aggregate
                if agg_method == 'FedAvg':
                    global_state = fedavg_aggregate(client_models, client_sizes)
                elif agg_method == 'FedProx':
                    global_state = fedprox_aggregate(client_models, client_sizes)
                elif agg_method == 'q-FedAvg':
                    global_state = qfedavg_aggregate(client_models, client_losses)
                elif agg_method == 'SCAFFOLD':
                    global_state = scaffold_aggregate(client_models, client_sizes)
                elif agg_method == 'FedAdam':
                    global_state = fedadam_aggregate(client_models, client_sizes)
                
                global_model.load_state_dict(global_state)
            
            # Evaluate
            accuracy, f1 = evaluate_model(global_model, X_val_scaled, y_val)
            all_accuracies.append(accuracy)
            all_f1s.append(f1)
            
            if fold_idx == N_SPLITS - 1:
                print(f"    Run {run_idx + 1}, Fold {fold_idx + 1}: Acc={accuracy:.4f}")
    
    # Statistics
    acc_mean = np.mean(all_accuracies)
    acc_std = np.std(all_accuracies, ddof=1)
    acc_min = np.min(all_accuracies)
    acc_max = np.max(all_accuracies)
    f1_mean = np.mean(all_f1s)
    f1_std = np.std(all_f1s, ddof=1)
    
    config_key = f"diabetes_{MODEL_TYPE}_{agg_method}"
    fl_results[config_key] = {
        'dataset': 'diabetes',
        'model': MODEL_TYPE,
        'aggregation': agg_method,
        'accuracy': {'mean': acc_mean, 'std': acc_std, 'min': acc_min, 'max': acc_max},
        'f1': {'mean': f1_mean, 'std': f1_std},
        'all_accuracies': all_accuracies,
        'all_f1s': all_f1s
    }
    
    print(f"\n  ✓ {agg_method} Results:")
    print(f"    Accuracy: {acc_mean*100:.2f}% ± {acc_std*100:.2f}% (range: {acc_min*100:.2f}% - {acc_max*100:.2f}%)")
    print(f"    F1-Score: {f1_mean*100:.2f}% ± {f1_std*100:.2f}%")

print("\n" + "="*80)
print("FL CONTINUATION COMPLETE")
print("="*80)


FEDERATED LEARNING - DIABETES FNN CONTINUATION

Results will be saved to: /kaggle/working/models_fl_diabetes_continue

Configuration: Diabetes FNN - 3 aggregation methods
  Remaining: ['q-FedAvg', 'SCAFFOLD', 'FedAdam']

  Aggregation: q-FedAvg
    Run 1, Fold 5: Acc=0.8656
    Run 2, Fold 5: Acc=0.8656
    Run 3, Fold 5: Acc=0.8650
    Run 4, Fold 5: Acc=0.8644
    Run 5, Fold 5: Acc=0.8652

  ✓ q-FedAvg Results:
    Accuracy: 86.46% ± 0.08% (range: 86.32% - 86.61%)
    F1-Score: 83.71% ± 0.17%

  Aggregation: SCAFFOLD
    Run 1, Fold 5: Acc=0.8650
    Run 2, Fold 5: Acc=0.8652
    Run 3, Fold 5: Acc=0.8643
    Run 4, Fold 5: Acc=0.8643
    Run 5, Fold 5: Acc=0.8657

  ✓ SCAFFOLD Results:
    Accuracy: 86.44% ± 0.10% (range: 86.22% - 86.62%)
    F1-Score: 83.74% ± 0.15%

  Aggregation: FedAdam
    Run 1, Fold 5: Acc=0.8647
    Run 2, Fold 5: Acc=0.8647
    Run 3, Fold 5: Acc=0.8644


In [ ]:
# ==================== SAVE RESULTS ====================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

results_json = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'dataset': 'diabetes',
        'model': 'FNN',
        'random_seeds': RANDOM_SEEDS,
        'n_splits': N_SPLITS,
        'total_evaluations': len(RANDOM_SEEDS) * N_SPLITS,
        'note': 'Continuation - q-FedAvg, SCAFFOLD, FedAdam only'
    },
    'federated_learning': fl_results
}

json_path = os.path.join(models_dir, 'fl_diabetes_fnn_continue.json')
with open(json_path, 'w') as f:
    json.dump(results_json, f, indent=2, default=lambda x: float(x) if isinstance(x, np.floating) else x)
print(f"✓ Saved: fl_diabetes_fnn_continue.json")

# Summary CSV
summary_data = []
for config_key, config_data in fl_results.items():
    summary_data.append({
        'Model': config_data['model'],
        'Aggregation': config_data['aggregation'],
        'Accuracy': config_data['accuracy']['mean'] * 100,
        'Std': config_data['accuracy']['std'] * 100,
        'Min': config_data['accuracy']['min'] * 100,
        'Max': config_data['accuracy']['max'] * 100,
        'F1': config_data['f1']['mean'] * 100
    })

summary_df = pd.DataFrame(summary_data)
csv_path = os.path.join(models_dir, 'fl_diabetes_fnn_summary.csv')
summary_df.to_csv(csv_path, index=False)
print(f"✓ Saved: fl_diabetes_fnn_summary.csv")

print("\n" + summary_df.to_string(index=False))

print("\n" + "="*80)
print("ALL RESULTS SAVED")
print("="*80)

## Summary

This notebook completed:

1. **Diabetes FNN - 3 aggregation methods**: q-FedAvg, SCAFFOLD, FedAdam
2. **5-fold CV × 5 runs** = 25 evaluations per method (75 total)
3. **Results saved** to `/kaggle/working/models_fl_diabetes_continue/`

**Next steps:**
- Download `fl_diabetes_fnn_continue.json`
- Merge with main FL results
- Run comprehensive analysis